In [ ]:
# Standard Library imports
from pathlib import Path

# External imports
import cv2
import matplotlib.pyplot as plt
import numpy as np
from machinevisiontoolbox.Camera import CentralCamera
from spatialmath import SE3, SO3

# Local imports
from slam import Processor

np.set_printoptions(suppress=True, precision=6)

Show pre and post-multiplication

In [ ]:
# Origin
R0 = np.eye(3)
t0 = np.array([0, 0, 0])
T0 = SE3.Rt(R0, t0)
origin = CentralCamera(pose=T0)
origin.plot(scale=0.25, color="black", frame=True)


# A new reference frame, start at x=-1
Rb = np.eye(3)
tb = np.array([-1, 0, 0])
Tb = SE3.Rt(Rb, tb)
camerab = CentralCamera(pose=Tb)
camerab.plot(scale=0.25, color="blue", frame=True)


# Pre-multiplication
R = SO3.Rz(90, unit="deg")
t = np.array([1, 0, 0])
T = SE3.Rt(R, t)
camerab.pose = T @ camerab.pose
camerab.plot(scale=0.25, color="green", frame=True)


# Post-multiplication
camerab.pose = camerab.pose @ T
ax = camerab.plot(scale=0.25, color="green", frame=True)


ax.view_init(elev=-90., azim=0, roll=-90)

In [3]:
K = np.array(
    [[2934.000267, 0.0, 1989.171435],
    [0.0, 2935.840316, 948.409835],
    [0.0, 0.0, 1.0]]
)

distortion_coeffs = np.array([[ 0.120889, -0.131111,  0.003076,  0.001933, -0.324229]])

In [4]:
class Photos:
    def __init__(self, base_path, extension="*.jpg"):
        self.files = list(Path(base_path).rglob(extension))
        self.is_opened = True
        self.image_generator = self._image_generator()

    def isOpened(self):
        return self.is_opened

    def release(self):
        pass

    def _image_generator(self):
        """Private generator method to yield images."""
        for impath in self.files:
            image = cv2.imread(str(impath))
            yield image

    def read(self):
        try:
            return True, next(self.image_generator)
        except StopIteration:
            return False, None

In [ ]:
ds = Photos("./images")

frame_ok, image = ds.read()
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.imshow(image)
plt.show()


frame_ok, image = ds.read()
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.imshow(image)
plt.show()

In [ ]:
options={"nfeatures":1000}
processor = Processor(K, distortion_coeffs, "sift", **options)
cap = Photos("./images")


# A pose is the extrinsic matrix, a 3x4 matrix
# Expressed in the world frame
initial_pose = np.array([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 1, 0]
], dtype=np.float32)


pose = initial_pose.copy()

while cap.isOpened():

    frame_ok, frame = cap.read()

    if not frame_ok:
        break

    preprocessed_frame = processor.preprocess_frame(frame, resize=True)
    kps, des = processor.detect_and_compute(preprocessed_frame)

    if processor.prev["frame"] is None:
        processor.prev = {"frame": preprocessed_frame, "kps": kps, "des": des}
        continue

    prev_matched_pts, curr_matched_pts = processor.get_matched_points(kps, des)

    composite = processor.show_matches(
        preprocessed_frame, prev_matched_pts, curr_matched_pts, cvshow=False
    )
    plt.imshow(composite)

    R, t = processor.pose_estimation_2d(prev_matched_pts, curr_matched_pts)
    T = SE3.Rt(R, t)

    # In the code that I've seen, the transform is inverted and post-multiplied, and I don't understand why, either of 
    # them
    # A post-multiplication will produce a translation and then a rotation
    T_inv = T.inv().A
    pose = pose @ T_inv

In [7]:
# I need to explain that the Essential Matrix can't be based on planar points

In [ ]:
K = processor.K

# Notice that imagesize takes (w,h) instead of what I'm used to provide (h,w)
camera = CentralCamera(
    name="camera 1", f=(K[0,0], K[1,1]), imagesize=(processor.w, processor.h), rho=1, pose=T_inv
)
camera

In [ ]:
# T0 is used as a reference frame, the origin
R0 = np.eye(3)
t0 = np.array([0, 0, 0])
T0 = SE3.Rt(R0, t0)

# Plot the initial pose (the origin)
origin = CentralCamera(pose=T0)
origin.plot(scale=0.25, color="black", frame=True)


# Start in a difference place than the origin
R1 = SO3.Rz(0, unit="deg")
t1 = np.array([0, 1, 0])
T1 = SE3.Rt(R1, t1)
camera = CentralCamera(pose=T1)
camera.plot(scale=0.25, color="red", frame=True)


# Plot the transformed pose (pre-multiplied)
pose = T.inv() @ T1
camera = CentralCamera(pose=pose)
camera.plot(scale=0.25, color="blue", frame=True)


# GREEN IS THE CORRECT ONE IN MY EXPERIMENT!
# Plot the transformed pose (post-multiplied)
pose = T1 @ T.inv()
camera = CentralCamera(pose=pose)
ax = camera.plot(scale=0.25, color="green", frame=True)


ax.view_init(elev=-90., azim=0, roll=-90)

# Now, with the correct answer at hand, the question is why post-multiplying it and why taking the inverse

#### The pose or the inverse of the pose?


In [ ]:
# T0 is the reference frame, the origin
R0 = np.eye(3)
t0 = np.array([0, 0, 0])
T0 = SE3.Rt(R0, t0)


camera = CentralCamera(pose=T0)
camera.plot(scale=0.25, color="black", frame=True)


# Plot the pose
camera = CentralCamera(pose=T)
ax = camera.plot(scale=0.25, color="red", frame=True)


# Plot the inverted pose
camera = CentralCamera(pose=T.inv())
ax = camera.plot(scale=0.25, color="green", frame=True)


ax.view_init(elev=-90., azim=0, roll=-90)

# From here, I conclude that recoverPose gives me the pose of Frame A expressed in Frame B.
# That's because the inverted pose gives me the correct result: the frame B expressed in Frame A: T_A_B, 
# hence, inverting the pose gives the pose of Frame A expressed in Frame B: T_B_A

From the graph above, I conclude that `recoverPose` gives me the pose of Frame A expressed in Frame B: `T_B_A`

That's because:
  - The inverted pose procues the correct result in my experiment: the Frame B expressed in Frame A: `T_A_B` (green frame)
  - hence, the pose before invertion was Frame A expressed in Frame B: `T_B_A` (red frame)

I want to reconciliate the documentation with my understanding. The documentation says:
        
    "(...) this matrix makes up a tuple that performs a change of basis from the first camera's coordinate system to the second camera's coordinate system."

So, it produces an operator that allows one to re-express a vector in a different frame.

In [ ]:
T_B_A = T
T_A_B = T_B_A.inv()
T_A_B.A @ np.array([[0, 0, 0, 1]]).T